Predicting Runs Scored: Data Wrangling

Last notebook we decided to switch our focus onto predicting runs scored
This could be useful in several capacities:

    - For predicting the over/under

    - To create features for predicting the game winner


We will focus on predicting the distribution of runs scored - that is, putting a probability on each possible value of runs scored for a team (up to some maximum value)

Predicting the distribution of a numeric target variable is known as "probabilistic regression"

In this notebook, we will "wrangle" our data such that each row represents a hitting team against a pitching team
The goal will be to predict the (distribution of the) number of runs scored by the hitting team

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [2]:
df = pd.read_csv('df_bp9.csv', low_memory=False)

Create df for runs of a single team

    - Want to "split" each game into two rows:

        - One where the home team bats against the visiting pitchers (and we try to predict the runs of the home team)

        - Other where we switch the roles of home and visiting

        - Then we can model how many runs we expect based on the hitters for that team and the opposing pitching

In [3]:
cols_ref = ['season', 'date','dblheader_code','date_dblhead','team_h','team_v',
            'league_h','league_v','over_under_line','over_under_result',
           'ballpark_id', 'day_night']

target_cols = ['runs_h', 'runs_v']

In [4]:
team_hit_stems = ['BATAVG','OBP','SLG','OBS','ERR','SB','CS']
lineup_hit_stems = ['BATAVG','OBP','SLG','OBS','SLGmod','SObat_perc']
strt_pitch_stems = ['ERA','WHIP','SO_perc','H_BB_perc','TB_BB_perc','FIP','FIP_perc']
bpen_pitch_stems = ['WHIP','SO_perc','H_BB_perc','TB_BB_perc']

team_hit_winsizes = [30,162]
lineup_hit_winsizes = [30,75,162,350]
strt_pitch_winsizes = [10,35,75]
bpen_pitch_winsizes = [10,35,75]

In [5]:
# We will make two data frames:
# version 'a': home hitting and visitor pitching
# version 'b': visit hitting and home pitching

team_hit_features_a = [x+'_'+str(winsize)+'_h' for winsize in team_hit_winsizes for x in team_hit_stems ]
lineup_hit_features_a = ['lineup'+n89+'_'+x+'_'+str(winsize)+wornot+'_h' for winsize in lineup_hit_winsizes
                        for x in lineup_hit_stems for wornot in ['','_w'] for n89 in ['8','9']]
start_pitch_features_a = ['Strt_'+x+'_'+str(winsize)+'_v' for winsize in strt_pitch_winsizes for x in strt_pitch_stems]
bpen_pitch_features_a = ['Bpen_'+x+'_'+str(winsize)+'_v' for winsize in bpen_pitch_winsizes for x in bpen_pitch_stems]

team_hit_features_b = [x+'_'+str(winsize)+'_v' for winsize in team_hit_winsizes for x in team_hit_stems ]
lineup_hit_features_b = ['lineup'+n89+'_'+x+'_'+str(winsize)+wornot+'_v' for winsize in lineup_hit_winsizes
                        for x in lineup_hit_stems for wornot in ['','_w'] for n89 in ['8','9']]
start_pitch_features_b = ['Strt_'+x+'_'+str(winsize)+'_h' for winsize in strt_pitch_winsizes for x in strt_pitch_stems]
bpen_pitch_features_b = ['Bpen_'+x+'_'+str(winsize)+'_h' for winsize in bpen_pitch_winsizes for x in bpen_pitch_stems]

In [6]:
cols_a = cols_ref + ['runs_h'] + team_hit_features_a + lineup_hit_features_a + start_pitch_features_a + bpen_pitch_features_a
df_a = df.loc[:,cols_a]
df_a['home_hitting'] = 1


cols_b = cols_ref + ['runs_v'] + team_hit_features_b + lineup_hit_features_b + start_pitch_features_b + bpen_pitch_features_b
df_b = df.loc[:,cols_b]
df_b['home_hitting'] = 0

stripped_feats  = [x[:-2] for x in team_hit_features_a + lineup_hit_features_a + 
                                              start_pitch_features_a + bpen_pitch_features_a]

final_col_list = cols_ref +['runs'] + stripped_feats+ ['home_hitting']

df_a.columns = final_col_list
df_b.columns = final_col_list
df_runs = pd.concat((df_a,df_b))

In [7]:
df_runs.sample(10)

,season,date,dblheader_code,date_dblhead,team_h,team_v,league_h,league_v,over_under_line,over_under_result,ballpark_id,day_night,runs,BATAVG_30,OBP_30,SLG_30,OBS_30,ERR_30,SB_30,CS_30,BATAVG_162,OBP_162,SLG_162,OBS_162,ERR_162,SB_162,CS_162,lineup8_BATAVG_30,lineup9_BATAVG_30,lineup8_BATAVG_30_w,lineup9_BATAVG_30_w,lineup8_OBP_30,lineup9_OBP_30,lineup8_OBP_30_w,lineup9_OBP_30_w,lineup8_SLG_30,lineup9_SLG_30,lineup8_SLG_30_w,lineup9_SLG_30_w,lineup8_OBS_30,lineup9_OBS_30,lineup8_OBS_30_w,lineup9_OBS_30_w,lineup8_SLGmod_30,lineup9_SLGmod_30,lineup8_SLGmod_30_w,lineup9_SLGmod_30_w,lineup8_SObat_perc_30,lineup9_SObat_perc_30,lineup8_SObat_perc_30_w,lineup9_SObat_perc_30_w,lineup8_BATAVG_75,lineup9_BATAVG_75,lineup8_BATAVG_75_w,lineup9_BATAVG_75_w,lineup8_OBP_75,lineup9_OBP_75,lineup8_OBP_75_w,lineup9_OBP_75_w,lineup8_SLG_75,lineup9_SLG_75,lineup8_SLG_75_w,lineup9_SLG_75_w,lineup8_OBS_75,lineup9_OBS_75,lineup8_OBS_75_w,lineup9_OBS_75_w,lineup8_SLGmod_75,lineup9_SLGmod_75,lineup8_SLGmod_75_w,lineup9_SLGmod_75_w,lineup8_SObat_perc_75,lineup9_SObat_perc_75,lineup8_SObat_perc_75_w,lineup9_SObat_perc_75_w,lineup8_BATAVG_162,lineup9_BATAVG_162,lineup8_BATAVG_162_w,lineup9_BATAVG_162_w,lineup8_OBP_162,lineup9_OBP_162,lineup8_OBP_162_w,lineup9_OBP_162_w,lineup8_SLG_162,lineup9_SLG_162,lineup8_SLG_162_w,lineup9_SLG_162_w,lineup8_OBS_162,lineup9_OBS_162,lineup8_OBS_162_w,lineup9_OBS_162_w,lineup8_SLGmod_162,lineup9_SLGmod_162,lineup8_SLGmod_162_w,lineup9_SLGmod_162_w,lineup8_SObat_perc_162,lineup9_SObat_perc_162,lineup8_SObat_perc_162_w,lineup9_SObat_perc_162_w,lineup8_BATAVG_350,lineup9_BATAVG_350,lineup8_BATAVG_350_w,lineup9_BATAVG_350_w,lineup8_OBP_350,lineup9_OBP_350,lineup8_OBP_350_w,lineup9_OBP_350_w,lineup8_SLG_350,lineup9_SLG_350,lineup8_SLG_350_w,lineup9_SLG_350_w,lineup8_OBS_350,lineup9_OBS_350,lineup8_OBS_350_w,lineup9_OBS_350_w,lineup8_SLGmod_350,lineup9_SLGmod_350,lineup8_SLGmod_350_w,lineup9_SLGmod_350_w,lineup8_SObat_perc_350,lineup9_SObat_perc_350,lineup8_SObat_perc_350_w,lineup9_SObat_perc_350_w,Strt_ERA_10,Strt_WHIP_10,Strt_SO_perc_10,Strt_H_BB_perc_10,Strt_TB_BB_perc_10,Strt_FIP_10,Strt_FIP_perc_10,Strt_ERA_35,Strt_WHIP_35,Strt_SO_perc_35,Strt_H_BB_perc_35,Strt_TB_BB_perc_35,Strt_FIP_35,Strt_FIP_perc_35,Strt_ERA_75,Strt_WHIP_75,Strt_SO_perc_75,Strt_H_BB_perc_75,Strt_TB_BB_perc_75,Strt_FIP_75,Strt_FIP_perc_75,Bpen_WHIP_10,Bpen_SO_perc_10,Bpen_H_BB_perc_10,Bpen_TB_BB_perc_10,Bpen_WHIP_35,Bpen_SO_perc_35,Bpen_H_BB_perc_35,Bpen_TB_BB_perc_35,Bpen_WHIP_75,Bpen_SO_perc_75,Bpen_H_BB_perc_75,Bpen_TB_BB_perc_75,home_hitting
92421,2006,20060407,0,200604070,TOR,TBA,AL,AL,0.0,NaN,TOR02,N,8,0.262646,0.321397,0.392996,0.714393,18.0,18.0,8.0,0.266523,0.324926,0.408203,0.733129,96.0,73.0,34.0,0.259635,0.258956,0.259527,0.258944,0.327562,0.324792,0.327349,0.324952,0.403605,0.394754,0.404271,0.396480,0.731167,0.719545,0.731620,0.721431,0.382606,0.372258,0.385801,0.376458,0.161914,0.161468,0.164529,0.163885,0.266852,0.266576,0.267028,0.266770,0.327361,0.326044,0.327466,0.326306,0.426049,0.413193,0.426485,0.415220,0.753409,0.739236,0.753951,0.741526,0.383524,0.368243,0.385959,0.372382,0.156173,0.153675,0.158451,0.156050,0.274015,0.268701,0.273556,0.268961,0.341332,0.334047,0.340850,0.334537,0.438543,0.422884,0.438608,0.424933,0.779875,0.756931,0.779457,0.759469,0.394303,0.378966,0.396988,0.383338,0.153345,0.150234,0.155631,0.152694,0.260858,0.258416,0.262414,0.260131,0.322739,0.318187,0.324494,0.320351,0.413712,0.402193,0.416110,0.405821,0.736451,0.720380,0.740604,0.726172,0.366277,0.356355,0.370222,0.361178,0.144100,0.145142,0.146839,0.147483,7.000000,1.722222,0.151751,0.361868,0.552529,6.611111,1.389105,4.995842,1.415800,0.174825,0.317483,0.447552,4.390852,0.984615,5.815385,1.531731,0.176957,0.338001,0.499045,5.128846,1.131763,1.584270,0.115385,0.361538,0.576923,1.740181,0.104723,0.394251,0.531828,1.510260,0.138132,0.357977,0.504864,1
56853,1990,19900604,0,199006040,NYN,MON,NL,NL,0.0,NaN,NYC17,N,5,0.261172,0.333930,0.404171,0.738101,19.0,49

In [8]:
df_runs.to_csv('df_runs_bp11.csv', index=False)